In [1]:
pip install --quiet langfuse langchain_openai langchain_community

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

AZURE_OPENAI_API_KEY = "61bf1b6851924c80ad18edeaade9cc9a"
AZURE_OPENAI_ENDPOINT = "https://eastus.api.cognitive.microsoft.com/"

os.environ['AZURE_OPENAI_API_KEY'] = AZURE_OPENAI_API_KEY
os.environ['AZURE_OPENAI_ENDPOINT'] = AZURE_OPENAI_ENDPOINT

os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-50abecd0-3a0d-42db-9214-c323065da23a"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-d25a8f0c-cd1b-49ec-b933-1ad31842b12e"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" #"http://localhost:3000"



In [ ]:
from langfuse import Langfuse
from langfuse.callback import CallbackHandler

# Initialize Langfuse client (prompt management)
langfuse = Langfuse()

# Initialize Langfuse CallbackHandler for Langchain (tracing)
langfuse_callback_handler = CallbackHandler()

# Optional, verify that Langfuse is configured correctly
assert langfuse.auth_check()
assert langfuse_callback_handler.auth_check()

In [5]:
lf = Langfuse(public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
              secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
              host=os.getenv("LANGFUSE_HOST", "http://localhost:3000"))

In [6]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='finance-gpt-v2')

In [7]:
company_name = input ('please mention the name of the company: ')

In [8]:
# Create a chat prompt
getting_ticker_prompt = lf.create_prompt(
    name="get_company_ticker_v2",
    prompt="You are a financial expert, get the ticker name for the {company_name} user have mentioned. Please return only the ticker and nothing else",

    labels=["production"],  # directly promote to production
    config={
        # "model": model,
        "temperature": 0.7,
        "supported_languages": ["en", "fr"],
    },  # optionally, add configs (e.g. model parameters or model tools) or tags
)

In [9]:
from langchain_core.output_parsers import StrOutputParser

# Create parser
parser = StrOutputParser()

In [10]:
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
tool = YahooFinanceNewsTool()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
import re
def clean_and_truncate(text, max_length=100):
    # Remove extra whitespace (newlines, tabs, multiple spaces)
    cleaned = re.sub(r'\s+', ' ', text).strip()
    # Return only the first `max_length` characters
    return cleaned[:max_length]

In [13]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [14]:
response_schemas = [
    ResponseSchema(name="company_name", description="The name of the company"),
    ResponseSchema(name="stock_code", description="The stock code of the company"),
    ResponseSchema(name="Sentiment", description="The sentiment of the news, Positive, Negative, Neutral"),
    ResponseSchema(name="People_names", description="The names of the people mentioned in the news"),
    ResponseSchema(name="Places_names", description="The names of the places mentioned in the news"),
    ResponseSchema(name="Other_companies_referred", description="The names of the companies mentioned in the news"),
    ResponseSchema(name="related_industries", description="The industries mentioned in the news"),
    ResponseSchema(name="market_implications", description="The market implications mentioned in the news"),
    ResponseSchema(name="confidence_score", description="The confidence score of the news", type = "float"),
]

In [15]:
final_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = final_output_parser.get_format_instructions()

In [16]:
# Create a chat prompt
getting_sentiment_prompt = lf.create_prompt(
    name="get_news_sentiment_v2",
    prompt="""You are a financial analyst. Analyze the following {text_input} about a company and extract the requested
    information in the follwoing JSON format.{format_instructions}""",
    labels=["production"],  # directly promote to production
    config={
        # "model": model,
        "temperature": 0.7,
        "supported_languages": ["en", "fr"],
    },  # optionally, add configs (e.g. model parameters or model tools) or tags
)


In [17]:
from langchain_core.runnables import RunnableLambda

format_sentiment_input = RunnableLambda(
    lambda text_input: {

        "text_input": text_input,
        "format_instructions": format_instructions
    }
)

In [18]:
ticker_prompt = lf.get_prompt("get_company_ticker_v2")
sentiment_prompt = lf.get_prompt("get_news_sentiment_v2")

In [19]:
from langchain_core.prompts import ChatPromptTemplate

ticker_prompt_langchain = ChatPromptTemplate.from_template(
        ticker_prompt.get_langchain_prompt(),
        metadata={"ticker_prompt": ticker_prompt},
    )

sentiment_prompt_langchain = ChatPromptTemplate.from_template(
        sentiment_prompt.get_langchain_prompt(),
        metadata={"sentiment_prompt": sentiment_prompt},
    )

In [20]:
chain = ticker_prompt_langchain | model | parser | tool | clean_and_truncate | format_sentiment_input | sentiment_prompt_langchain | model | final_output_parser

In [ ]:
chain.invoke({"company_name": company_name})